<a href="https://colab.research.google.com/github/Shona1709/ML/blob/Python-scripts/GA_extended_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sklearn-genetic

     |████████████████████████████████| 160 kB 6.8 MB/s 


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualization library
import matplotlib.pyplot as plt
import time
from subprocess import check_output
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


import sklearn
from google.colab import files, drive, data_table
from google.colab.data_table import *
from lightgbm import *
from sklearn.ensemble import *
from sklearn.ensemble import *
from sklearn.neural_network import *
from sklearn.model_selection import *
from sklearn.naive_bayes import *
from xgboost import *
from sklearn.svm import *
from sklearn.linear_model import *
from genetic_selection import GeneticSelectionCV
from sklearn.metrics import *

from sklearn.tree import *
import warnings
from sklearn.utils import *
from yellowbrick.model_selection import *
from yellowbrick.classifier import *
import matplotlib.pyplot as plt
#from bayes_opt import *
#from mlxtend.feature_selection import *
import warnings
warnings.filterwarnings("ignore")
#import h2o
##from h2o.automl import H2OAutoML
##h2o.init()

mcc = make_scorer(matthews_corrcoef)

In [ ]:
data = pd.read_csv('breast_cancer_final.csv');

In [ ]:
y = data['diagnosis']                   # M or B
list = ['id','diagnosis']
x = data.drop(list,axis = 1 )
y = y.replace("B", 0)
y = y.replace("M", 1)


In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(x)
X_scale

array([[0.52103744, 0.0226581 , 0.54598853, ..., 0.91202749, 0.59846245,
        0.41886396],
       [0.64314449, 0.27257355, 0.61578329, ..., 0.63917526, 0.23358959,
        0.22287813],
       [0.60149557, 0.3902604 , 0.59574321, ..., 0.83505155, 0.40370589,
        0.21343303],
       ...,
       [0.45525108, 0.62123774, 0.44578813, ..., 0.48728522, 0.12872068,
        0.1519087 ],
       [0.64456434, 0.66351031, 0.66553797, ..., 0.91065292, 0.49714173,
        0.45231536],
       [0.03686876, 0.50152181, 0.02853984, ..., 0.        , 0.25744136,
        0.10068215]])

In [ ]:
estimator = LogisticRegression(solver = "liblinear", C = 6, tol = 1, fit_intercept = True)

from sklearn.model_selection import *
report = pd.DataFrame()
nofeats = []
chosen_feats = []
cvscore = []
rkf = RepeatedStratifiedKFold(n_repeats = 20, n_splits = 5)
for i in range(2,11):

  selector = GeneticSelectionCV(estimator,
                                cv = rkf,
                                verbose = 0,
                                scoring = mcc,
                                max_features = i,
                                n_population = 200,
                                crossover_proba = 0.5,
                                mutation_proba = 0.2,
                                n_generations = 10,
                                crossover_independent_proba=0.5,
                                mutation_independent_proba=0.1,
                                #tournament_size = 3,
                                n_gen_no_change=10,
                                caching=True,
                                n_jobs=-1)
  selector = selector.fit(x, y)
  genfeats = x.columns[selector.support_]
  #genfeats = list(genfeats)
  print("Chosen Feats:  ", genfeats)

  cv_score = selector.generation_scores_[-1]
  nofeats.append(len(genfeats))
  chosen_feats.append(genfeats)
  cvscore.append(cv_score)
report["No of Feats"] = nofeats
report["Chosen Feats"] = chosen_feats
report["Scores"] = cvscore


Chosen Feats:   Index(['radius_worst', 'concavity_worst'], dtype='object')
Chosen Feats:   Index(['radius_worst', 'compactness_worst', 'concavity_worst'], dtype='object')
Chosen Feats:   Index(['radius_worst', 'compactness_worst', 'concavity_worst',
       'concave points_worst'],
      dtype='object')
Chosen Feats:   Index(['concavity_se', 'radius_worst', 'compactness_worst', 'concavity_worst',
       'concave points_worst'],
      dtype='object')
Chosen Feats:   Index(['compactness_mean', 'concavity_mean', 'fractal_dimension_mean',
       'radius_worst', 'concavity_worst', 'concave points_worst'],
      dtype='object')
Chosen Feats:   Index(['concavity_mean', 'concave points_mean', 'smoothness_se',
       'radius_worst', 'concavity_worst', 'concave points_worst'],
      dtype='object')
Chosen Feats:   Index(['compactness_mean', 'concavity_mean', 'concave points_mean',
       'symmetry_se', 'radius_worst', 'concavity_worst',
       'concave points_worst'],
      dtype='object')
Chosen

In [ ]:
report["Scores"] = np.round(report["Scores"], 3)
report.sort_values(by = "Scores", ascending = False, inplace = True)
#report.index
ga_feats = report.iloc[0]["Chosen Feats"]
DataTable(report)
print(ga_feats)

Index(['compactness_mean', 'concavity_mean', 'fractal_dimension_mean',
       'radius_worst', 'concavity_worst', 'concave points_worst'],
      dtype='object')


In [ ]:
report

,No of Feats,Chosen Feats,Scores
4,6,"Index(['compactness_mean', 'concavity_mean', '...",0.889
5,6,"Index(['concavity_mean', 'concave points_mean'...",0.889
6,7,"Index(['compactness_mean', 'concavity_mean', '...",0.889
7,6,"Index(['concavity_mean', 'concave points_mean'...",0.889
8,5,"Index(['compactness_mean', 'concavity_mean', '...",0.888
3,5,"Index(['concavity_se', 'radius_worst', 'compac...",0.886
2,4,"Index(['radius_worst', 'compactness_worst', 'c...",0.885
1,3,"Index(['radius_worst', 'compactness_worst', 'c...",0.883
0,2,"Index(['radius_worst', 'concavity_worst'], dty...",0.871


In [ ]:
estimator = RandomForestClassifier(max_depth = 3)
from sklearn.model_selection import *
report = pd.DataFrame()
nofeats = []
chosen_feats = []
cvscore = []
rkf = RepeatedStratifiedKFold(n_repeats = 1, n_splits = 10)
for i in range(2,11):

  selector = GeneticSelectionCV(estimator,
                                cv = rkf,
                                verbose = 0,
                                scoring = "roc_auc",
                                max_features = i,
                                n_population = 30,
                                crossover_proba = 0.5,
                                mutation_proba = 0.2,
                                n_generations = 10,
                                crossover_independent_proba=0.5,
                                mutation_independent_proba=0.05,
                                tournament_size = 1,
                                n_gen_no_change = 5,
                                caching = True,
                                n_jobs=-1)
  selector = selector.fit(x, y)

  print(selector.support_)

  genfeats = x.columns[selector.support_]
  #genfeats = list(genfeats)
  print(genfeats)

 # X = D[genfeats] ; y = D.iloc[:,-1]
  #clf = selector.estimator

  cv_score = selector.generation_scores_[-1]
  nofeats.append(len(genfeats))
  chosen_feats.append(genfeats)
  cvscore.append(cv_score)

report["No of Feats"] = nofeats
report["Chosen Feats"] = chosen_feats
report["Scores"] = np.round(cvscore, 3)

[False False False  True False False False False False False False False
 False False False False False False False False False False False False
 False False False  True False False]
Index(['area_mean', 'concave points_worst'], dtype='object')
[False False  True False  True False False False False False False False
 False False False False False False False False False  True False False
 False False False False False False]
Index(['perimeter_mean', 'smoothness_mean', 'texture_worst'], dtype='object')
[False False False False False False False  True False False False False
 False False False False False  True False False False False False  True
  True False False False False False]
Index(['concave points_mean', 'concave points_se', 'area_worst',
       'smoothness_worst'],
      dtype='object')
[False False False False False False False  True False False False False
 False False False False  True False False False  True  True False False
 False False  True False False False]
Index(['co

In [ ]:
report.sort_values(by = "Scores", ascending = False, inplace = True)
report

,No of Feats,Chosen Feats,Scores
6,7,"Index(['concavity_mean', 'perimeter_se', 'conc...",0.991
7,5,"Index(['concave points_mean', 'radius_se', 'co...",0.991
3,5,"Index(['concave points_mean', 'concavity_se', ...",0.990
4,4,"Index(['area_se', 'area_worst', 'concavity_wor...",0.990
5,4,"Index(['area_mean', 'concave points_mean', 'ra...",0.990
8,10,"Index(['radius_mean', 'texture_mean', 'perimet...",0.990
2,4,"Index(['concave points_mean', 'concave points_...",0.985
0,2,"Index(['area_mean', 'concave points_worst'], d...",0.983
1,3,"Index(['perimeter_mean', 'smoothness_mean', 't...",0.980


In [ ]:
report["Scores"] = np.round(report["Scores"], 3)
report.sort_values(by = "Scores", ascending = False, inplace = True)
#report.index
ga_feats = report.iloc[0]["Chosen Feats"]
DataTable(report)
print(ga_feats)

Index(['concavity_mean', 'perimeter_se', 'concave points_se', 'texture_worst',
       'area_worst', 'smoothness_worst', 'symmetry_worst'],
      dtype='object')
